## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). 

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

In [1]:
import pandas as pd
import requests

df = pd.read_csv('SINASC_RO_2019.csv')
print(df.shape)
df.drop_duplicates().shape

# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# Conte o número de valores missing por variável.

pd.set_option('display.max_rows', 69) #alterando quantidade de itens que são exibidos pelo Pandas

df.isna().sum() # somando os itens de cada coluna que tem dados ausentes

ORIGEM              0
CODESTAB          115
CODMUNNASC          0
LOCNASC             0
IDADEMAE            0
ESTCIVMAE         317
ESCMAE            312
CODOCUPMAE       2907
QTDFILVIVO       1573
QTDFILMORT       2098
CODMUNRES           0
GESTACAO         1232
GRAVIDEZ           79
PARTO              49
CONSULTAS           0
DTNASC              0
HORANASC           21
SEXO                4
APGAR1             96
APGAR5            103
RACACOR           647
PESO                0
IDANOMAL          591
DTCADASTRO          0
CODANOMAL       26814
NUMEROLOTE          0
VERSAOSIST          0
DTRECEBIM           0
DIFDATA             0
DTRECORIGA      27028
NATURALMAE        298
CODMUNNATU        298
CODUFNATU         298
ESCMAE2010        249
SERIESCMAE      12710
DTNASCMAE          40
RACACORMAE        661
QTDGESTANT       1212
QTDPARTNOR       1879
QTDPARTCES       1791
IDADEPAI        19421
DTULTMENST      10072
SEMAGESTAC       1238
TPMETESTIM       1238
CONSPRENAT        930
MESPRENAT 

In [3]:
pd.reset_option('display.max_rows') # retornando configuração padrão

In [4]:
#colunas a serem analisadas
colunas_analise = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

#criando um novo DataFrame somente com as colunas que serem analisadas
df_analise = df[colunas_analise].copy()

df_analise.head(2)                

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0


In [5]:
# Quantidade de linhas com a coluna 'APGAR5' com dado ausente
df_analise['APGAR5'].isna().sum()

103

In [6]:
# Quantidade de linhas no DataFrame
df_analise.shape

(27028, 9)

In [7]:
# Quantidade de dados ausentes em cada coluna
df_analise.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [8]:
# Retirando todas as linhas com a variável resposta sem estar respondida
df_analise = df_analise.dropna(subset=['APGAR5'])

In [9]:
# Quantidade de dados ausentes
df_analise.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [10]:
print(27028 - 103)
print(df_analise.shape)

26925
(26925, 9)


In [11]:
# Dados únicos na coluna 'ESTCIVMAE'
df_analise['ESTCIVMAE'].unique()

array([ 5.,  2.,  9.,  1.,  4., nan,  3.])

In [12]:
# Dados únicos na coluna 'CONSULTAS'
df_analise['CONSULTAS'].unique()

array([4, 3, 2, 1, 9], dtype=int64)

In [13]:
# preenchendo com 9.0 os dados ausentes na coluna 'ESTCIVMAE'
df_analise['ESTCIVMAE'].fillna(9.0, inplace=True)

In [14]:
# Verificação
print(df_analise['ESTCIVMAE'].unique())
print(df_analise['ESTCIVMAE'].isnull().sum())

[5. 2. 9. 1. 4. 3.]
0


In [15]:
# Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.

df_analise['QTDFILVIVO'].unique()

array([ 0.,  1.,  2.,  3., nan,  4.,  6.,  5.,  7., 12.,  9., 11.,  8.,
       30., 10., 14.])

In [16]:
# preenchendo com 0.0 os dados ausentes na coluna 'QTDFILVIVO'
df_analise['QTDFILVIVO'].fillna(0.0, inplace=True)

In [17]:
# Verificação
df_analise['QTDFILVIVO'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  6.,  5.,  7., 12.,  9., 11.,  8., 30.,
       10., 14.])

In [18]:
df_analise.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

Variáveis que permanecem com valores nulos:
- ESCMAE (Escolaridade, anos de estudo concluídos) 
- GESTACAO (Semanas de gestação)
- GRAVIDEZ (Tipo de gravidez: Única, Dupla, Trípla e mais)

### Análise da coluna "ESCMAE"

In [19]:
# distribuição dos dados na coluna 'ESCMAE'
df_analise['ESCMAE'].value_counts()

8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: ESCMAE, dtype: int64

In [20]:
print(15581 / 26925 * 100)
print(310 / 26925 * 100)

57.86815227483751
1.1513463324048283


- Conforme informações do IBGE 2022 "https://educa.ibge.gov.br/jovens/conheca-o-brasil/populacao/18317-educacao.html", 
53% das pessoas com 25 anos ou mais de idade possuim no mínimo o ensino médio completo.
Em nossa base de dados, 8 a 11 anos de escolaridade se refere no mínimo que a pessoa finalizou o ensino fundamental.
- Sendo assim assim, nossa base de dados está coerente com os dados do censo.
- Os dados nulos esta coluna se refere somente a 1%
- Iremos considerar os dados nulos como "8 a 11 anos"


### Análise da coluna "GESTACAO"

In [21]:
# distribuição dos dados na coluna 'GESTACAO'
df_analise['GESTACAO'].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [22]:
df_analise['GESTACAO'].value_counts().sum()

25709

In [23]:
print(22523 / 26925 * 100)
# neste base de dados, gravidez com 37 a 41 semanas corresponde a 83% 
print(1216 / 26925 * 100)
# neste base de dados, dados nulos nesta coluna corresponde a 4% 

83.65088207985144
4.516248839368616


**Uma gestação "normal" denominado tecnicamente de "a termo", tem um período de 37 a 41 semanas. 
Iremos considerar dados nulos sendo "37 a 41 semanas".**

### Análise da coluna "GRAVIDEZ"

In [24]:
# distribuição dos dados na coluna 'GRAVIDEZ'
df_analise['GRAVIDEZ'].value_counts()

Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

**Iremos considerar que os dados nulos são referente à gravidez única, pois é raro uma mulher ter uma gravidez de gêmeos, seja monozigóticos ou dizigóticos.**

---> Limpeza dos dados

In [25]:
df_analise['ESCMAE'].fillna('8 a 11 anos', inplace=True)

In [26]:
df_analise['GESTACAO'].fillna('37 a 41 semanas', inplace=True)

In [27]:
df_analise['GRAVIDEZ'].fillna('Única', inplace=True)

In [28]:
# Verificação
df_analise.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [29]:
# Criando um novo Data_Frame, conservando as últimos modificações no DataFrame 'df_analise'
df_definitivo = df_analise.copy()

In [30]:
df_definitivo['APGAR5'].unique()

array([10.,  9.,  7.,  0.,  8.,  4.,  3.,  6.,  5.,  2.,  1.])

In [31]:
# Distribuição dos dados
df_definitivo['APGAR5'].value_counts()

9.0     16076
10.0     8393
8.0      1994
7.0       241
6.0        79
5.0        45
1.0        24
4.0        23
2.0        18
0.0        17
3.0        15
Name: APGAR5, dtype: int64

In [32]:
# Categorização da variável 'APGAR5'
df_definitivo.loc[(df_definitivo['APGAR5'] >= 0.0) & (df_definitivo['APGAR5'] <= 3.0), 'STATUSASFIX'] = 'asfixia severa'
df_definitivo.loc[(df_definitivo['APGAR5'] >= 4.0) & (df_definitivo['APGAR5'] <= 5.0), 'STATUSASFIX'] = 'asfixia moderada'
df_definitivo.loc[(df_definitivo['APGAR5'] >= 6.0) & (df_definitivo['APGAR5'] <= 7.0), 'STATUSASFIX'] = 'asfixia leve'
df_definitivo.loc[(df_definitivo['APGAR5'] >= 8.0) & (df_definitivo['APGAR5'] <= 10.0), 'STATUSASFIX'] = 'normal'

In [33]:
# Não há dados nulos na variável 'STATUSASFIX'
df_definitivo['STATUSASFIX'].unique()

array(['normal', 'asfixia leve', 'asfixia severa', 'asfixia moderada'],
      dtype=object)

In [34]:
# Verificação
df_definitivo.loc[df_definitivo['APGAR5'] == 6.0, :]

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,STATUSASFIX
205,1,28,2.0,4 a 7 anos,2.0,37 a 41 semanas,Única,3,6.0,asfixia leve
289,1,25,5.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,6.0,asfixia leve
407,1,29,2.0,12 anos ou mais,0.0,28 a 31 semanas,Única,3,6.0,asfixia leve
449,1,24,2.0,12 anos ou mais,0.0,28 a 31 semanas,Única,2,6.0,asfixia leve
544,1,37,1.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,6.0,asfixia leve
...,...,...,...,...,...,...,...,...,...,...
26620,1,28,2.0,12 anos ou mais,1.0,22 a 27 semanas,Única,4,6.0,asfixia leve
26637,1,23,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,6.0,asfixia leve
26728,1,31,5.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,6.0,asfixia leve
26893,1,21,5.0,1 a 3 anos,4.0,37 a 41 semanas,Única,2,6.0,asfixia leve
